In [1]:
from google.colab import drive
drive.mount('/content/drive/')

%cd /content/drive/MyDrive/Job_Clausthal/TNTM/Code/Preprocessing

!pip install transformers
!pip install octis

Mounted at /content/drive/
/content/drive/MyDrive/Job_Clausthal/TNTM/Code/Preprocessing
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 91.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 86.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 KB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.6/170.6 KB 18.7 MB/s eta 0:00:00
  Preparing metadata (setup.p

In [2]:
%load_ext autoreload
%autoreload 2


In [3]:
import pickle
from Preprocessor import Preprocessor
import torch
from transformers import BertModel
from tqdm import tqdm 

In [4]:
torch.manual_seed(41)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'current device: {device}')

current device: cuda


In [5]:
data_raw = pickle.load(open("../../Data/Preprocessed_Data/octis_dataset_reuters.pickle", "rb"))

In [6]:
from transformers import BertTokenizer

# bert-base-uncased model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# tokenizer for every single word
tokenizer_word = lambda string: tokenizer.encode_plus(
    text = string,
    add_special_tokens = False
)['input_ids']

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
bert1 = BertModel.from_pretrained("bert-base-uncased")
bert1.to(device)
bert1.eval()
bert1.requires_grad = False

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
bert_encoder = Preprocessor.encode_bert_wrap(bert1)

In [9]:
corpus_raw_str_lis = [" ".join(doc) for doc in data_raw.get_corpus()]
prep = Preprocessor(corpus_raw_str_lis)

In [10]:
res = prep.compute_embeddings(
    tokenizer_word = tokenizer_word, 
    transformer_chunk_len = 512 - 2,
    encoder = bert_encoder, 
    device = device, 
)

tokenizing...
tokenizing...


100%|██████████| 10788/10788 [00:00<00:00, 235435.70it/s]


compute embeddings of entire corpus...


100%|██████████| 10788/10788 [00:10<00:00, 1002.10it/s]


compute mean embeddings of each individual word...


/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:6393: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().sort_index(
100%|██████████| 1329600/1329600 [02:09<00:00, 10243.09it/s]


compute bow representation of each document...


100%|██████████| 10788/10788 [00:12<00:00, 859.05it/s]


In [11]:
df, word_df, cleaned_bow_worddata= res

In [12]:
word_df.sort_values(by = "word", inplace = True)

In [13]:
with open("../../Data/DataResults_BERT/cleaned_bow_worddata_REUTERS_BERT.pickle", "wb") as file: 
  pickle.dump(cleaned_bow_worddata, file)

In [14]:
with open("../../Data/DataResults_BERT/cleaned_embedding_df_REUTERS_BERT.pickle", "wb") as file: 
  pickle.dump(word_df, file)